In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import datetime
from itertools import product

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

**Tensorboard und Funktionen**

In [2]:
log_dir_model_no = "logs/fit/model_no/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir_model_public = "logs/fit/model_public/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir_model_school = "logs/fit/model_school/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Erstellen Sie einen TensorBoard-Callback
tensorboard_callback_model_no = TensorBoard(log_dir=log_dir_model_no, histogram_freq=1)
tensorboard_callback_model_public = TensorBoard(log_dir=log_dir_model_public, histogram_freq=1)
tensorboard_callback_model_school = TensorBoard(log_dir=log_dir_model_school, histogram_freq=1)

In [3]:
def calculate_mmad_values(column):
    
    median = column.median()
    mad = np.median(np.abs(column - median))
    return median, mad

In [4]:
def mmad_denormalize(normalized_value, median, mad):
    return normalized_value * mad + median

In [5]:
def preprocessing(df):
    features = ['revenue', 'revenuePY', 'revenue-1', 'revenue-2', 'revenue-3', 'revenuePY-1', 'revenuePY-2', 'revenuePY-3', 'revenuePY+1', 'revenuePY+2', 'revenue+2']

    median, mad = calculate_mmad_values(df['revenue'])
    
    # Globale Normalisierung durchführen
    for feature in features:
        df[feature] = (df[feature] - median) / mad
    
          
          
    #Input Daten
    X_revenue = df[['revenue-3', 'revenue-2', 'revenue-1', 'revenue', 'filiale_baeckerei']]
    X_revenuePY = df[['revenuePY-3', 'revenuePY-2', 'revenuePY-1', 'revenuePY', 'revenuePY+1', 'revenuePY+2', 'filiale_baeckerei']]
    X_holidays = df[['carnival', 'easter', 'ascension_day', 'whitsunday', 'christmas', 'new_year', 'filiale_baeckerei']]
    Y = df[['revenue+2', 'filiale_baeckerei']]  # Zielvariable

    return X_revenue, X_revenuePY, X_holidays, Y, median, mad

In [6]:
def compute_split_indices(df):
    """
    Diese Funktion berechnet einmal die Split-Indizes basierend auf der Spalte 'filiale_baeckerei'.
    """
    # 1. Extrahiere die einzigartigen Kombinationen aus 'filiale_baeckerei'
    unique_combinations = df['filiale_baeckerei'].unique()

    # 2. Teile die Kombinationen in 70% Training, 15% Validierung und 15% Test auf
    train_combinations, test_combinations = train_test_split(unique_combinations, test_size=0.30, random_state=42)  # 70% Training, 30% Test+Val
    val_combinations, test_combinations = train_test_split(test_combinations, test_size=0.5, random_state=42)  # 50% der 30% für Val/Test
    
    return train_combinations, val_combinations, test_combinations


In [7]:
def apply_split(X_revenue, X_revenuePY, X_holidays, Y, train_indices, val_indices, test_indices):
    """
    Diese Funktion wendet die Split-Indizes auf die jeweiligen Datensets an und formatiert sie entsprechend.
    Zunächst wird der Index der DataFrames synchronisiert, um sicherzustellen, dass die Reihenfolge stimmt.
    """
    train_indices = X_revenue['filiale_baeckerei'].isin(train_combinations)
    val_indices = X_revenue['filiale_baeckerei'].isin(val_combinations)
    test_indices = X_revenue['filiale_baeckerei'].isin(test_combinations)
    
    # Synchronisiere die Indizes der DataFrames, um sicherzustellen, dass sie übereinstimmen
    X_revenue = X_revenue.reset_index(drop=True)
    X_revenuePY = X_revenuePY.reset_index(drop=True)
    X_holidays = X_holidays.reset_index(drop=True)
    Y = Y.reset_index(drop=True)

    # Extrahiere Trainings-, Validierungs- und Testsets mithilfe der Indizes und wende .loc an
    X_revenue_train = X_revenue.loc[train_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 4, 1)
    X_revenuePY_train = X_revenuePY.loc[train_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 6, 1)
    X_holidays_train = X_holidays.loc[train_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 6, 1)
    Y_train = Y.loc[train_indices].drop(columns=['filiale_baeckerei']).values  # Eindimensionales Ziel (Y)
    Y_train = Y_train.flatten()

    X_revenue_val = X_revenue.loc[val_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 4, 1)
    X_revenuePY_val = X_revenuePY.loc[val_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 6, 1)
    X_holidays_val = X_holidays.loc[val_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 6, 1)
    Y_val = Y.loc[val_indices].drop(columns=['filiale_baeckerei']).values  # Eindimensionales Ziel (Y)
    Y_val = Y_val.flatten()

    X_revenue_test = X_revenue.loc[test_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 4, 1)
    X_revenuePY_test = X_revenuePY.loc[test_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 6, 1)
    X_holidays_test = X_holidays.loc[test_indices].drop(columns=['filiale_baeckerei']).values.reshape(-1, 6, 1)
    Y_test = Y.loc[test_indices].drop(columns=['filiale_baeckerei']).values  # Eindimensionales Ziel (Y)
    Y_test = Y_test.flatten()
    
    # Verifizierung der Größen der Splits
    print("Trainingsdaten:")
    print("X_revenue_train:", X_revenue_train.shape)
    print("X_revenuePY_train:", X_revenuePY_train.shape)
    print("X_holidays_train:", X_holidays_train.shape)
    print("Y_train:", Y_train.shape)

    print("\nValidierungsdaten:")
    print("X_revenue_val:", X_revenue_val.shape)
    print("X_revenuePY_val:", X_revenuePY_val.shape)
    print("X_holidays_val:", X_holidays_val.shape)
    print("Y_val:", Y_val.shape)

    print("\nTestdaten:")
    print("X_revenue_test:", X_revenue_test.shape)
    print("X_revenuePY_test:", X_revenuePY_test.shape)
    print("X_holidays_test:", X_holidays_test.shape)
    print("Y_test:", Y_test.shape)

    return (X_revenue_train, X_revenuePY_train, X_holidays_train, Y_train, 
            X_revenue_val, X_revenuePY_val, X_holidays_val, Y_val, 
            X_revenue_test, X_revenuePY_test, X_holidays_test, Y_test)

In [8]:
# Deine Modell-Definition als Funktion
def build_model(num_layers, neurons_per_layer, learning_rate):
    # Inputs
    input_revenue = Input(shape=(4, 1), name='input_current_year_revenue')
    input_revenuePY = Input(shape=(6, 1), name='input_previous_year_revenue')
    input_holidays = Input(shape=(6, 1), name='input_public_holidays')

    # Flatten-Schichten für die Sequencen
    flat_revenue = Flatten(name='flatten_current_year_revenue')(input_revenue)
    flat_revenuePY = Flatten(name='flatten_previous_year_revenue')(input_revenuePY)
    flat_holidays = Flatten(name='flatten_public_holidays')(input_holidays)

    # Zusammenführen der Outputs durch ein Concatenate-Layer
    concat = concatenate([flat_revenue, flat_revenuePY, flat_holidays], name='concat_layer')

    # Anzahl der Layer (1 bis 4) und Neuronen pro Layer (4, 8, 12, 16, 20)
    x = concat
    for i in range(num_layers):
        x = Dense(neurons_per_layer, activation='relu', name=f'dense_layer_{i+1}')(x)

    # Dense Layer für die Vorhersage
    output = Dense(1, name='output_layer')(x)

    # Modell erstellen
    model = Model(inputs=[input_revenue, input_revenuePY, input_holidays], outputs=output)

    # Kompilieren des Modells
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=['mae'])  # MAE wird als Metrik verwendet

    return model

In [9]:
def Hyperparamteroptimization(X_revenue_train, X_revenuePY_train, X_holidays_train, Y_train, 
                              X_revenue_val, X_revenuePY_val, X_holidays_val, Y_val):
    # Definiere alle möglichen Werte für die Hyperparameter
    num_layers_options = [1,2,3,4]  # Anzahl der Schichten
    neurons_per_layer_options = [10,20,30,40]  # Neuronen pro Schicht
    learning_rate_options = [0.01, 0.001, 0.0001]  # Lernrate
    batch_size_options = [16, 32, 64, 128]  # Batchgrößen

    # Erstelle alle Kombinationen der Hyperparameter
    combinations = list(product(num_layers_options, neurons_per_layer_options, 
                                learning_rate_options, batch_size_options))
    
    # Anzahl der Kombinationen für Fortschrittsanzeige
    total_combinations = len(combinations)

    # Manuelle Schleife über alle Kombinationen
    best_mae = float('inf')
    best_combination = None

    # Fortschrittszähler
    for idx, (num_layers, neurons_per_layer, learning_rate, batch_size) in enumerate(combinations):
        # Anzeige des Fortschritts
        print(f"Testing combination {idx + 1}/{total_combinations}: Layers={num_layers}, Neurons={neurons_per_layer}, LR={learning_rate}, Batch={batch_size}")

        # Baue das Modell mit der aktuellen Kombination
        model = build_model(num_layers, neurons_per_layer, learning_rate)

        # Trainiere das Modell
        history = model.fit(
            [X_revenue_train, X_revenuePY_train, X_holidays_train], Y_train,
            epochs=150,  # Anzahl der Epochen
            batch_size=batch_size,
            validation_data=([X_revenue_val, X_revenuePY_val, X_holidays_val], Y_val),
            callbacks=[EarlyStopping(monitor='val_mae', patience=10)]
        )

        # Überprüfe den besten Validation MAE
        val_mae = min(history.history['val_mae'])
        if val_mae < best_mae:
            best_mae = val_mae
            best_combination = (num_layers, neurons_per_layer, learning_rate, batch_size)

        # Anzeige des aktuellen besten MAE
        print(f"Current best MAE: {best_mae}")

    # Beste Kombination ausgeben
    print(f"Beste Kombination: Layers={best_combination[0]}, Neurons={best_combination[1]}, LR={best_combination[2]}, Batch={best_combination[3]}")
    print(f"Bester Validation MAE: {best_mae}")

    return best_combination[0], best_combination[1], best_combination[2], best_combination[3]

 # **Dense with no shift**

In [10]:
datei_pfad = '../data/no_shift_forecasting.csv'
df_no = pd.read_csv(datei_pfad)

In [11]:
X_revenue_no, X_revenuePY_no, X_holidays_no, Y_no, median_no, mad_no = preprocessing(df_no)

In [12]:
train_combinations = np.load('train_combinations.npy', allow_pickle=True)
val_combinations = np.load('val_combinations.npy', allow_pickle=True)
test_combinations = np.load('test_combinations.npy', allow_pickle=True)

In [13]:
X_revenue_train_no, X_revenuePY_train_no, X_holidays_train_no, Y_train_no, X_revenue_val_no, X_revenuePY_val_no, X_holidays_val_no, Y_val_no, X_revenue_test_no, X_revenuePY_test_no, X_holidays_test_no, Y_test_no = apply_split(X_revenue_no, X_revenuePY_no, X_holidays_no, Y_no, train_combinations, val_combinations, test_combinations)

Trainingsdaten:
X_revenue_train: (35375, 4, 1)
X_revenuePY_train: (35375, 6, 1)
X_holidays_train: (35375, 6, 1)
Y_train: (35375,)

Validierungsdaten:
X_revenue_val: (7860, 4, 1)
X_revenuePY_val: (7860, 6, 1)
X_holidays_val: (7860, 6, 1)
Y_val: (7860,)

Testdaten:
X_revenue_test: (7790, 4, 1)
X_revenuePY_test: (7790, 6, 1)
X_holidays_test: (7790, 6, 1)
Y_test: (7790,)


**Find best Model**

In [14]:
num_layers_no, num_neurons_no, learning_rate_no, batch_size_no = Hyperparamteroptimization(X_revenue_train_no, X_revenuePY_train_no, X_holidays_train_no, Y_train_no, X_revenue_val_no, X_revenuePY_val_no, X_holidays_val_no, Y_val_no)

Testing combination 1/192: Layers=1, Neurons=10, LR=0.01, Batch=16
Epoch 1/150
2211/2211 [==============================] - 5s 2ms/step - loss: 0.1170 - mae: 0.2328 - val_loss: 0.1011 - val_mae: 0.2337
Epoch 2/150
 323/2211 [===>..........................] - ETA: 2s - loss: 0.0975 - mae: 0.2199

KeyboardInterrupt: 

In [ ]:
# Print der Werte der Hyperparameter
print(f"Number of Layers (No): {num_layers_no}")
print(f"Number of Neurons per Layer (No): {num_neurons_no}")
print(f"Learning Rate (No): {learning_rate_no}")
print(f"Batch Size (No): {batch_size_no}")

**train final model**

In [ ]:
# Baue das Modell mit den besten Hyperparametern
model_dense_no = build_model(num_layers_no, num_neurons_no, learning_rate_no)

# Trainiere das Modell
history = model_dense_no.fit(
    [X_revenue_train_no, X_revenuePY_train_no, X_holidays_train_no], Y_train_no,
    epochs=150,  # Anzahl der Epochen
    batch_size=batch_size_no,
    validation_data=([X_revenue_val_no, X_revenuePY_val_no, X_holidays_val_no], Y_val_no),
    callbacks=[EarlyStopping(monitor='val_mae', patience=15),
              tensorboard_callback_model_no]
)

**Mean Relativ Error Testset**

In [ ]:
# Vorhersagen für die Testdaten
y_pred_no = model_dense_no.predict([X_revenue_test_no, X_revenuePY_test_no, X_holidays_test_no])
y_pred_no = y_pred_no.flatten()
original_Y_no = mmad_denormalize(Y_test_no, median_no, mad_no)
original_Y_pred_no = mmad_denormalize(y_pred_no, median_no, mad_no)

In [ ]:
# Berechnung des relativen Fehlers
abs_erros_no = original_Y_no - original_Y_pred_no
relative_errors_no = (original_Y_no - original_Y_pred_no) / original_Y_no

# Berechnung des durchschnittlichen relativen Fehlers
mean_relative_error_no = np.mean(np.abs(relative_errors_no))
print(f'Mean Relative Error No Shift Testset: {mean_relative_error_no}')

**Mean Relativ Error Trainset**

In [ ]:
# Vorhersagen für die Testdaten
y_pred_no = model_dense_no.predict([X_revenue_train_no, X_revenuePY_train_no, X_holidays_train_no])
y_pred_no = y_pred_no.flatten()
original_Y_no = mmad_denormalize(Y_train_no, median_no, mad_no)
original_Y_pred_no = mmad_denormalize(y_pred_no, median_no, mad_no)

In [ ]:
# Berechnung des relativen Fehlers
abs_erros_no = original_Y_no - original_Y_pred_no
relative_errors_no = (original_Y_no - original_Y_pred_no) / original_Y_no

# Berechnung des durchschnittlichen relativen Fehlers
mean_relative_error_no = np.mean(np.abs(relative_errors_no))
print(f'Mean Relative Error No Shift Trainset: {mean_relative_error_no}')

# **Dense with public shift**

In [ ]:
datei_pfad = '../data/public_shift_forecasting.csv'
df_public = pd.read_csv(datei_pfad)

In [ ]:
X_revenue_public, X_revenuePY_public, X_holidays_public, Y_public, median_public, mad_public = preprocessing(df_public)

In [ ]:
X_revenue_train_public, X_revenuePY_train_public, X_holidays_train_public, Y_train_public, X_revenue_val_public, X_revenuePY_val_public, X_holidays_val_public, Y_val_public, X_revenue_test_public, X_revenuePY_test_public, X_holidays_test_public, Y_test_public = apply_split(X_revenue_public, X_revenuePY_public, X_holidays_public, Y_public, train_combinations, val_combinations, test_combinations)

In [ ]:
num_layers_public, num_neurons_public, learning_rate_public, batch_size_public = Hyperparamteroptimization(X_revenue_train_public, X_revenuePY_train_public, X_holidays_train_public, Y_train_public, X_revenue_val_public, X_revenuePY_val_public, X_holidays_val_public, Y_val_public)

In [ ]:
num_layers_public = 2
num_neurons_public = 40
learning_rate_public = 0.001
batch_size_public = 64

In [ ]:
# Print der Werte der Hyperparameter
print(f"Number of Layers (Public): {num_layers_public}")
print(f"Number of Neurons per Layer (Public): {num_neurons_public}")
print(f"Learning Rate (Public): {learning_rate_public}")
print(f"Batch Size (Public): {batch_size_public}")

In [ ]:

# Bauemodel_dense_public das Modell mit den besten Hyperparametern
model_dense_public = build_model(num_layers_public, num_neurons_public, learning_rate_public)

# Trainiere das Modell
history = model_dense_public.fit(
    [X_revenue_train_public, X_revenuePY_train_public, X_holidays_train_public], Y_train_public,
    epochs=150,  # Anzahl der Epochen
    batch_size=batch_size_public,
    validation_data=([X_revenue_val_public, X_revenuePY_val_public, X_holidays_val_public], Y_val_public),
    callbacks=[EarlyStopping(monitor='val_mae', patience=15),
               tensorboard_callback_model_public]
)

**Testset**

In [ ]:
# Vorhersagen für die Testdaten
y_pred_public = model_dense_public.predict([X_revenue_test_public, X_revenuePY_test_public, X_holidays_test_public])
y_pred_public = y_pred_public.flatten()
original_Y_public = mmad_denormalize(Y_test_public, median_public, mad_public)
original_Y_public = original_Y_public.flatten()
original_Y_pred_public = mmad_denormalize(y_pred_public, median_public, mad_public)

In [ ]:
# Berechnung des relativen Fehlers
abs_erros_public = original_Y_public - original_Y_pred_public
relative_errors_public = (original_Y_public - original_Y_pred_public) / original_Y_public

# Berechnung des durchschnittlichen relativen Fehlers
mean_relative_error_public = np.mean(np.abs(relative_errors_public))
print(f'Mean Relative Error for public Shift Testset: {mean_relative_error_public}')

**Trainset**

In [ ]:
# Vorhersagen für die Testdaten
y_pred_public = model_dense_public.predict([X_revenue_train_public, X_revenuePY_train_public, X_holidays_train_public])
y_pred_public = y_pred_public.flatten()
original_Y_public = mmad_denormalize(Y_train_public, median_public, mad_public)
original_Y_public = original_Y_public.flatten()
original_Y_pred_public = mmad_denormalize(y_pred_public, median_public, mad_public)

In [ ]:
# Berechnung des relativen Fehlers
abs_erros_public = original_Y_public - original_Y_pred_public
relative_errors_public = (original_Y_public - original_Y_pred_public) / original_Y_public

# Berechnung des durchschnittlichen relativen Fehlers
mean_relative_error_public = np.mean(np.abs(relative_errors_public))
print(f'Mean Relative Error for public Shift Testset: {mean_relative_error_public}')

# **Dense with school shift**

In [ ]:
datei_pfad = '../data/school_shift_forecasting.csv'
df_school = pd.read_csv(datei_pfad)

In [ ]:
X_revenue_school, X_revenuePY_school, X_holidays_school, Y_school, median_school, mad_school = preprocessing(df_school)

In [ ]:
X_revenue_train_school, X_revenuePY_train_school, X_holidays_train_school, Y_train_school, X_revenue_val_school, X_revenuePY_val_school, X_holidays_val_school, Y_val_school, X_revenue_test_school, X_revenuePY_test_school, X_holidays_test_school, Y_test_school = apply_split(X_revenue_school, X_revenuePY_school, X_holidays_school, Y_school, train_combinations, val_combinations, test_combinations)

In [ ]:
num_layers_school, num_neurons_school, learning_rate_school, batch_size_school = Hyperparamteroptimization(X_revenue_train_school, X_revenuePY_train_school, X_holidays_train_school, Y_train_school, X_revenue_val_school, X_revenuePY_val_school, X_holidays_val_school, Y_val_school, )

In [ ]:
# Print der Werte der Hyperparameter
print(f"Number of Layers (School): {num_layers_school}")
print(f"Number of Neurons per Layer (School): {num_neurons_school}")
print(f"Learning Rate (School): {learning_rate_school}")
print(f"Batch Size (School): {batch_size_school}")

In [ ]:
# Zusammenführen von Trainings- und Validierungsdaten
X_revenue_combined_school = np.concatenate([X_revenue_train_school, X_revenue_val_school], axis=0)
X_revenuePY_combined_school = np.concatenate([X_revenuePY_train_school, X_revenuePY_val_school], axis=0)
X_holidays_combined_school = np.concatenate([X_holidays_train_school, X_holidays_val_school], axis=0)
Y_combined_school = np.concatenate([Y_train_school, Y_val_school], axis=0)

# Baue das Modell mit den besten Hyperparametern
model_dense_school = build_model(num_layers_school, num_neurons_school, learning_rate_school)

# Trainiere das Modell
history = model_dense_school.fit(
    [X_revenue_train_school, X_revenuePY_train_school, X_holidays_train_school], Y_train_school,
    epochs=150,  # Anzahl der Epochen
    batch_size=batch_size_school,
    validation_data=([X_revenue_val_school, X_revenuePY_val_school, X_holidays_val_school], Y_val_school),
    callbacks=[EarlyStopping(monitor='val_mae', patience=15),
              tensorboard_callback_model_school]
)

In [ ]:
# Vorhersagen für die Testdaten
y_pred_school = model_dense_school.predict([X_revenue_test_school, X_revenuePY_test_school, X_holidays_test_school])
y_pred_school = y_pred_school.flatten()
original_Y_school = mmad_denormalize(Y_test_school, median_school, mad_school)
original_Y_pred_school = mmad_denormalize(y_pred_school, median_school, mad_school)

In [ ]:
# Berechnung des relativen Fehlers
abs_erros_school = original_Y_school - original_Y_pred_school
relative_errors_school = (original_Y_school - original_Y_pred_school) / original_Y_school

# Berechnung des durchschnittlichen relativen Fehlers
mean_relative_error_school = np.mean(np.abs(relative_errors_school))
print(f'Mean Relative Error School Shift Testset: {mean_relative_error_school}')

In [ ]:
# Vorhersagen für die Testdaten
y_pred_school = model_dense_school.predict([X_revenue_train_school, X_revenuePY_train_school, X_holidays_train_school])
y_pred_school = y_pred_school.flatten()
original_Y_school = mmad_denormalize(Y_train_school, median_school, mad_school)
original_Y_pred_school = mmad_denormalize(y_pred_school, median_school, mad_school)

In [ ]:
# Berechnung des relativen Fehlers
abs_erros_school = original_Y_school - original_Y_pred_school
relative_errors_school = (original_Y_school - original_Y_pred_school) / original_Y_school

# Berechnung des durchschnittlichen relativen Fehlers
mean_relative_error_school = np.mean(np.abs(relative_errors_school))
print(f'Mean Relative Error School Shift Trainset: {mean_relative_error_school}')

**Open Tensorboard**

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit